# **`Project 2: Team Thomas Allinson`**

### **Objective**: Analyze the comparative costs of a vegan diet versus an omnivorous diet within the American population, with a specific focus on their environmental impact.

#### Group Members:
> Johann: johann.dicken@berkeley.edu <br>
> Laure: laureho@berkeley.edu <br>
> Reily: reilyjean@berkeley.edu <br>
> Carmen: carmenvega@berkeley.edu <br>
> Steven: k1519632@berkeley.edu <br>

### **[A]: Description of population of interest**

...descripition here...

### **[A]: Dietary Reference Intakes**

In [6]:
import pandas as pd
import numpy as np

In [28]:
# Import Dietary Requirements spreadsheet data as a pd.DataFrame
diet_min = pd.read_csv('Dietary_Requirements.csv')
diet_min.head()

,Nutrition,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
0,Energy,---,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
1,Protein,RDA,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
2,"Fiber, total dietary",---,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
3,"Folate, DFE",RDA,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
4,"Calcium, Ca",RDA,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


Dietary function takes 2 arguments: `age`, a positive integer, and `sex`, a string (not case-senstitive) with the classification of male, female, or child.

In [29]:
def dietary_ref(age, sex):

    # Validate age input
    if not isinstance(age, int) or age <= 0:
        return "Incorrect age input. Please enter a positive integer for the age."
    
    # Normalize and validate sex input
    sex = sex.lower()
    if sex not in ['male', 'female', 'child']:
        return "Incorrect sex input. Input must be Male, Female, or Child."
    
    # Determine the appropriate column based on age and sex
    if sex == 'child':
        if age <= 3:
            col_name = 'C 1-3'
        elif age <= 8:
            col_name = 'C 4-8'
        else:
            return "Age out of range for child category."
    else:
        if age <= 8:
            col_name = f"{'F' if sex == 'female' else 'M'} 4-8"
        elif age <= 13:
            col_name = f"{'F' if sex == 'female' else 'M'} 9-13"
        elif age <= 18:
            col_name = f"{'F' if sex == 'female' else 'M'} 14-18"
        elif age <= 30:
            col_name = f"{'F' if sex == 'female' else 'M'} 19-30"
        elif age <= 50:
            col_name = f"{'F' if sex == 'female' else 'M'} 31-50"
        else:
            col_name = f"{'F' if sex == 'female' else 'M'} 51+"
    
    # Extract and return the relevant nutrient recommendations
    if col_name in diet_min.columns:
        return diet_min[['Nutrition', col_name]].set_index('Nutrition')[col_name]
    else:
        return "Matching column not found in DataFrame. Check the column names."

In [30]:
# Example usage
dietary_ref(15, 'Male')

Nutrition
Energy                            2200.0
Protein                             52.0
Fiber, total dietary                30.8
Folate, DFE                        400.0
Calcium, Ca                       1300.0
Carbohydrate, by difference        130.0
Iron, Fe                            11.0
Magnesium, Mg                      410.0
Niacin                              16.0
Phosphorus, P                     1250.0
Potassium, K                      4700.0
Riboflavin                           1.3
Thiamin                              1.2
Vitamin A, RAE                     900.0
Vitamin B-12                         2.4
Vitamin B-6                          1.3
Vitamin C, total ascorbic acid      75.0
Vitamin E (alpha-tocopherol)        15.0
Vitamin K (phylloquinone)           75.0
Zinc, Zn                            11.0
Name: M 14-18, dtype: float64

### **[A]: Data on prices for different foods**

Let's import our google spreadsheet as a pd.DataFrame here!

In [11]:
# prices_df = pd.read_csv('file_name.csv')
# prices_df
apikey = "KNqUDtV7Kcktiuheo3EoNhB0zDlCevFAdqZrKgdj" 
%pip install -r requirements.txt --upgrade
import fooddatacentral as fdc

  Using cached Pint-0.24.4-py3-none-any.whl.metadata (8.5 kB)
  Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached gspread-6.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached gspread_pandas-3.3.0-py2.py3-none-any.whl.metadata (10 kB)
  Using cached Bottleneck-1.4.2-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.7 kB)
  Using cached eep153_tools-0.12.4-py2.py3-none-any.whl.metadata (363 bytes)
  Using cached fooddatacentral-1.0.10-py3-none-any.whl.metadata (1.9 kB)
  Using cached python_gnupg-0.5.4-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached flexcache-0.3-py3-none-any.whl.metadata (7.0 kB)
  Using cached flexparser-0.4-py3-none-any.whl.metadata (18 kB)
Using cached Pint-0.24.4-py3-none-any.whl (302 kB)
Using cached scipy-1.15.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.6 MB)
Using cached gspread_pandas-3.3.0-py2.py3-none-any.w

In [18]:

import re

# Load the CSV file into a DataFrame
df = pd.read_csv('food_and_prices.csv')

# Define a regex pattern for common animal products
animal_product_pattern = r'\b(butter|cheese|milk|kefir|whey|eggnog|beef|chicken|dulce|pork|egg|fish|lamb|turkey|turtle|breast|mollusks|frog|thigh|yogurt|honey|gelatin|cream|lard|sausage|anchovy|shellfish|shrimp|mayo|ham|meat)\b'

# Create a new column 'animal product' that marks items based on the pattern
df['animal product'] = df['Food commodity ITEM'].apply(
    lambda x: 'animal product' if re.search(animal_product_pattern, str(x), re.IGNORECASE) else 'plant-based'
)

# Display the updated DataFrame
df.rename(columns={'Food commodity ITEM': 'Food'}, inplace=True)
df[df['animal product'] == 'plant-based'].head(100)


,Food,Carbon Footprint kg CO2eq/kg or l of food ITEM,Water Footprint liters water/kg o liter of food ITEM,FDC ID,FDC Food Name,Average Price per kg (USD),animal product
1,SIMPLE COOKIES**,1.39,1723.0,2707964,"Cookie, shortbread",10.80,plant-based
2,BREAD MULTICEREAL**,0.70,771.0,2707777,"Bread, multigrain",4.24,plant-based
3,BREAD PLAIN**,0.89,1031.0,174929,"Bread, sticks, plain",4.24,plant-based
4,BREAD WHOLE**,0.77,887.0,2707709,"Bread, whole wheat",4.24,plant-based
5,FLAVORED CRACKERS**,0.93,1378.0,2055556,FLAVORED CRACKERS,8.80,plant-based
...,...,...,...,...,...,...,...
107,LETTUCE,0.41,237.0,2709789,"Lettuce, raw",1.50,plant-based
108,SPINACH,0.34,292.0,1905313,SPINACH,2.20,plant-based
109,CARROT,0.24,195.0,2709660,"Carrots, raw",1.50,plant-based
110,GARLIC,0.71,589.0,1662203,GARLIC,10.00,plant-based


In [23]:
df.set_index('Food', inplace=True)

In [24]:
for food in df.index:
    print(food)

CHOCOLATE OR CREAM FILLED COOKIES**
SIMPLE COOKIES**
BREAD MULTICEREAL**
BREAD PLAIN**
BREAD WHOLE**
FLAVORED CRACKERS**
PLAIN CRACKERS**
WHOLEGRAIN CRACKERS**
CRISPBREAD**
KETCHUP
EGG PASTA*
ORANGE JUICE
OAT MEAL
CHOCOLATE
BEANS (F)
MARGARINE*
OLIVE OIL
PALM OIL
PEANUT OIL
RAPESEED OIL
SOYBEAN OIL
SUNFLOWER OIL
PASTA*
PESTO
PESTO WITHOUT GARLIC
BEET SUGAR
CANE SUGAR
CAKES AND CROISSANT**
COCOA CAKES AND CROISSANT**
TOMATO PEELED
TOMATO PUREE
TOMATO & BASIL
TOMATO ARRABBIATA
CORN CANNED
QUORNE
SOY BURGER
TOFU
YEAST COMPRESSED*
BEEF BONE FREE MEAT*
BUTTER*
CREAM
MASCARPONE
MOZZARELLA
RICOTTA
STRACCHINO
CHEESE
PARMIGIANO REGGIANO
MEALWORMS*
LAMB BONE FREE MEAT*
LAMB MEAT WITH BONE*
COW MILK
PORK BONE FREE MEAT*
BACON
CHICKEN BONE FREE MEAT
YOGURT WHITE
CLEMENTINE
LEMON
LIME
MANDARIN
ORANGE
TANGERIN
APPLE
APRICOT
AVOCADO
BANANA
FIG
GRAPES
GUAVA
KIWI
MANGO
PEACH
PEAR
PINEAPPLE
PLUM
WATERMELON
BLACKBERRY
BLUEBERRY
CRANBERRY
CURRANT
GOOSEBERRY
RASPBERRY
STRAWBERRY
BARLEY
MAIZE
MILLET
OAT
RYE

In [27]:
#Building A
import warnings
import fooddatacentral as fdc
#id = 2705854
#fdc.nutrients(apikey,fdc_id=id)
D = {}
count = 0


for food in df.index:
    try:
        FDC = df.loc[df.index==food,:]['FDC ID'][0]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
        #print(food)
    except AttributeError:
        warnings.warn(f"Couldn't find FDC Code {FDC} for food {food}.")

D = pd.DataFrame(D,dtype=float)

D


/tmp/ipykernel_72/3037320227.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  FDC = df.loc[df.index==food,:]['FDC ID'][0]


,CHOCOLATE OR CREAM FILLED COOKIES**,SIMPLE COOKIES**,BREAD MULTICEREAL**,BREAD PLAIN**,BREAD WHOLE**,FLAVORED CRACKERS**,PLAIN CRACKERS**,WHOLEGRAIN CRACKERS**,CRISPBREAD**,KETCHUP,...,EGGPLANT,PEPPER,PUMPKIN,TOMATO,ZUCCHINI,CARP,COD,SALMON,TROUT,TURBOT
Alanine,NaN,NaN,NaN,0.395,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.971
"Alcohol, ethyl",0.00,0.00,0.0,0.000,0.00,NaN,0.00,0.00,NaN,0.00,...,0.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,0.00,NaN
Amino acids,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000
Arginine,NaN,NaN,NaN,0.432,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.960
Ash,NaN,NaN,NaN,3.900,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vitamin K (Dihydrophylloquinone),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vitamin K (phylloquinone),2.40,11.00,1.4,2.200,7.80,NaN,69.30,36.00,NaN,3.00,...,24.30,NaN,NaN,NaN,NaN,4.00,NaN,NaN,12.00,NaN
Vitamins and Other Components,NaN,NaN,NaN,0.000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000
Water,4.50,3.60,36.9,6.100,38.70,NaN,3.14,2.50,NaN,68.50,...,70.40,NaN,NaN,NaN,NaN,60.50,NaN,NaN,53.20,76.950


In [48]:
#set up mins
#bmin = diet_min.set_index('Nutrition', inplace= True)

bmin = diet_min
bmin = bmin.drop('Source',axis=1)
#bmax
bmax = pd.read_csv('diet_max.csv')

bmax = bmax.drop('Source',axis=1) #
bmax = bmax.set_index('Nutrition')

bmin

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


In [51]:
Prices

Food
APPLE                     2.20
APRICOT                   3.00
ARTICHOKE                 4.40
AVOCADO                   4.40
BACON                    12.00
                         ...  
WHEAT                     4.24
WHOLEGRAIN CRACKERS**     8.80
YEAST COMPRESSED*         2.20
YOGURT WHITE              3.50
ZUCCHINI                  2.20
Name: Average Price per kg (USD), Length: 127, dtype: float64

In [52]:
#prices
df.dropna(how='any') # Drop food with any missing data

Prices = df.groupby('Food')['Average Price per kg (USD)'].min()
p = Prices.apply(lambda x:x).dropna()

# Compile list that we have both prices and nutritional info for; drop if either missing
use = p.index.intersection(D.columns)
p = p[use]

p

APPLE                     2.20
APRICOT                   3.00
ARTICHOKE                 4.40
AVOCADO                   4.40
BACON                    12.00
                         ...  
WHEAT                     4.24
WHOLEGRAIN CRACKERS**     8.80
YEAST COMPRESSED*         2.20
YOGURT WHITE              3.50
ZUCCHINI                  2.20
Name: Average Price per kg (USD), Length: 127, dtype: float64

In [53]:
# Drop nutritional information for foods we don't know the price of,
# and replace missing nutrients with zeros.
Aall = D[p.index].fillna(0)

# Drop rows of A that we don't have constraints for.
Amin = Aall.loc[bmin.index]

Amax = Aall.loc[bmax.index]

# Maximum requirements involve multiplying constraint by -1 to make <=.
A = pd.concat([Amin,-Amax])

A

,APPLE,APRICOT,ARTICHOKE,AVOCADO,BACON,BANANA,BARLEY,BEAN,BEANS (F),BEEF BONE FREE MEAT*,...,TOMATO PUREE,TROUT,TURBOT,TURNIP,WATERMELON,WHEAT,WHOLEGRAIN CRACKERS**,YEAST COMPRESSED*,YOGURT WHITE,ZUCCHINI
Nutrition,,,,,,,,,,,,,,,,,,,,,
Energy,52.00,53.00,53.000,160.000,600.00,312.00,139.000,119.000,119.000,261.000,...,25.00,271.000,397.000,90.000,31.00,274.000,482.000,439.00,62.00,21.00
Protein,0.00,0.00,2.890,2.000,33.33,12.50,2.200,5.440,5.440,25.600,...,1.67,17.100,16.050,2.460,0.78,10.700,7.100,8.40,3.52,1.05
"Fiber, total dietary",3.20,0.00,5.700,6.700,0.00,6.20,3.700,4.900,4.900,0.000,...,1.70,0.500,0.000,2.400,0.40,4.000,3.500,8.10,0.00,1.10
"Folate, DFE",0.00,0.00,89.000,81.000,0.00,0.00,16.000,21.000,21.000,10.000,...,0.00,45.000,8.000,41.000,0.00,99.000,143.000,785.00,0.00,0.00
"Calcium, Ca",0.00,0.00,21.000,12.000,0.00,125.00,11.000,26.000,26.000,24.000,...,12.00,34.000,18.000,114.000,0.00,125.000,14.000,19.00,132.00,21.00
"Carbohydrate, by difference",14.29,13.53,12.000,8.530,0.00,40.62,27.200,15.900,15.900,0.000,...,5.00,11.700,0.000,3.390,7.45,47.500,67.600,18.10,4.41,4.21
"Iron, Fe",0.23,0.00,0.610,0.550,0.00,1.12,1.280,1.600,1.600,2.470,...,1.67,1.130,0.360,1.630,0.00,3.600,2.590,3.25,0.00,0.44
"Magnesium, Mg",0.00,0.00,42.000,29.000,0.00,0.00,21.000,40.000,40.000,20.000,...,0.00,24.000,51.000,18.000,0.00,41.000,29.000,40.00,13.00,0.00
Niacin,0.00,0.00,1.110,1.740,0.00,0.00,1.990,0.430,0.430,5.070,...,0.00,5.260,2.200,0.388,0.00,5.590,3.400,12.30,0.00,0.00


In [54]:
b = pd.concat([bmin,-bmax]) # Note sign change for max constraints

b

,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,
Energy,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
"Fiber, total dietary",14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
"Folate, DFE",150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
"Calcium, Ca",700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0
"Carbohydrate, by difference",130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0,130.0
"Iron, Fe",7.0,10.0,10.0,8.0,8.0,15.0,11.0,18.0,8.0,18.0,8.0,8.0,8.0
"Magnesium, Mg",80.0,130.0,130.0,240.0,240.0,360.0,410.0,310.0,400.0,320.0,420.0,320.0,420.0
Niacin,6.0,8.0,8.0,12.0,12.0,14.0,16.0,14.0,16.0,14.0,16.0,14.0,16.0


In [55]:
from  scipy.optimize import linprog as lp
import numpy as np

tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros

## Choose sex/age group!
group = "F 19-30"

# Now solve problem!  (Note that the linear program solver we'll use assumes
# "less-than-or-equal" constraints.  We can switch back and forth by
# multiplying $A$ and $b$ by $-1$.)

result = lp(p, -A, -b[group], method='highs')

result

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 30.968972929091414
              x: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
            nit: 13
          lower:  residual: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
                 marginals: [ 1.827e+00  2.815e+00 ...  1.035e+00
                              1.215e+00]
          upper:  residual: [       inf        inf ...        inf
                                    inf]
                 marginals: [ 0.000e+00  0.000e+00 ...  0.000e+00
                              0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 1.100e+03  2.570e+01 ...  1.234e+03
                              0.000e+00]
                 marginals: [-0.000e+00 -0.000e+00 ... -0.000e+00
                             -1.930e-03]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip

In [56]:
print(f"Cost of diet for {group} is ${result.fun:.2f} per day.")

Cost of diet for F 19-30 is $30.97 per day.


In [57]:
# Put back into nice series
diet = pd.Series(result.x,index=p.index)

print("\nYou'll be eating (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.


You'll be eating (in 100s of grams or milliliters):
CARROT               0.371357
COW MILK             4.211071
COWPEA               2.536917
LETTUCE              1.074562
MANGO                0.309460
PEPPER               0.011320
SOY BURGER           1.179279
SUNFLOWER OIL        0.208774
YEAST COMPRESSED*    2.964912
dtype: float64


### **[A]: Nutritional content of different foods**

...

### **[A]: Solution**

I think it'd be cool to make a graph for this :) For example, an overlying bar graph with different colors for sex, going across the x-axis with ages, y-axis being minimum diet cost.

In [29]:
# Code here

### **[B]: Is your solution edible?**

...

### **[B]: What is total cost for population of interest?**

In [31]:
# Import wbdata
# Code function for total cost

### **[C]: Sensitivity of Solution**

In [30]:
# Code here